# Using Stan to solve resistance model

(c) 2021 Manuel Razo & Tom Röschinger. This work is licensed under a 
[Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). 
All code contained herein is licensed under an 
[MIT license](https://opensource.org/licenses/MIT).

***

In [15]:
using Distributed, Jedi, CairoMakie, CSV, StanSample, Glob, Statistics, ColorSchemes, Colors, Distributed
addprocs(2)

@everywhere begin
    using StanSample
    using DataFrames
    using Statistics
end

Jedi.styles.default_makie!()

First we try to use Stan to find the maximum growth rate in each growth curve. First we will use a Gaussian Process to do so.

In [5]:
resistance_model = "
functions {
   real poly(
       real K_d,
       real kappa_t,
       real j,
       real delta_r,
       real lambda_0,
       real K_M,
       real V_0,
       real a_ex,
       real lambda
   ){
       real c1 = K_d * delta_r * kappa_t;
       real a_3 = c1* (K_M * lambda_0 - c1 + V_0);
     
       vector[3] a_2;
       a_2[1] = K_M * j * a_ex * lambda_0^2;
       a_2[2] = c1^2 * (-j + 2*lambda_0);
       a_2[3] = c1 * lambda_0 * (K_M * j - K_M * lambda_0 - j * a_ex - V_0);

       real a_1 = c1 * lambda_0 * (-K_M * j * lambda_0 + c1 * (2 * j - lambda_0) + j * a_ex * lambda_0);

       real a_0 = -c1^2 * lambda_0^2 * j;

       return a_3 * lambda^3 + sum(a_2) * lambda^2 + a_1 * lambda + a_0; 
   }

   vector system(
        vector y,        // unknowns
        vector theta,    // parameters
        real[] x_r, // data (real)
        int[] x_i) {     // data (integer)
  vector[1] z;
  z[1] = poly(theta[1], theta[2], theta[3], theta[4], theta[5], theta[6], theta[7], theta[8], y[1]);
  return z;
}

}

data {
    real K_d;
    real kappa_t;
    real j;
    real delta_r;
    real lambda_0;
    real K_M;
    int N;
    real a_ex[N];
    real lambda[N];
    
}

transformed data {
  vector[1] y_guess;
  y_guess[1] = lambda_0;
  real x_r[0];
  int x_i[0];
}

parameters {
  real log_V_0;
}

transformed parameters {
  real V_0 = 10^log_V_0;

  vector[8] theta;
  theta[1] = K_d;
  theta[2] = kappa_t;
  theta[3] = j;
  theta[4] = delta_r;
  theta[5] = lambda_0;
  theta[6] = K_M;
  theta[7] = V_0;
  theta[8] = 0;
  vector[N] y;
  for (i in 1:N){
      theta[8] = a_ex[i];
      y[i] = algebra_solver_newton(system, y_guess, theta, x_r, x_i)[1];
  }
  
}


model {
    log_V_0 ~ normal(0, 3);
    

}
/*
generated quantities {
    real c;
    c = poly(K_d, kappa_t, j, delta_r, lambda_0, K_M, V_0, a_ex, y[1]);
}
*/

";

In [6]:
data = Dict(    
    "K_d" => 0.1,
    "kappa_t"=>0.06,
    "j" => 100,
    "delta_r" => 46.5,
    "lambda_0" =>0.68,
    "K_M" => 10,
    "V_0" => 0,
    "N" => 4,
    "a_ex" => [0, 1, 5, 10]
    
)

Dict{String, Any} with 9 entries:
  "K_M"      => 10
  "V_0"      => 0
  "delta_r"  => 46.5
  "a_ex"     => [0, 1, 5, 10]
  "lambda_0" => 0.68
  "j"        => 100
  "N"        => 4
  "K_d"      => 0.1
  "kappa_t"  => 0.06

In [16]:
sm = SampleModel("resistance_model", resistance_model)
rc = stan_sample(sm; data, num_chains=2);


/var/folders/lt/c_93j4d15l1bgn9mncp1y18r0000gn/T/jl_6vK81H/resistance_model.stan updated.


LoadError: 
Error when compiling SampleModel resistance_model:
make: *** [/var/folders/lt/c_93j4d15l1bgn9mncp1y18r0000gn/T/jl_6vK81H/resistance_model] Interrupt: 2


In [8]:
st = read_samples(sm)
st[Symbol("y.1")][1]

LoadError: UndefVarError: idx not defined

## Load Data

In [17]:
folder_list = [#'../processing/growth_curves_plate_reader/20210604_r1_MG1655_319_IW_tc/',
              # '../processing/growth_curves_plate_reader/20210606_r1_MG1655_319_IW__UV5_WT_tc/',
               #'../processing/growth_curves_plate_reader/20210608_r2_MG1655_319_IW_tc/',
    "../processing/growth_curves_plate_reader/20210712_r1_MG1655_319_IW_UV5_WT_tc/",
    "../processing/growth_curves_plate_reader/20210713_r2_MG1655_319_IW_UV5_WT_tc/",
    "../processing/growth_curves_plate_reader/20210714_r3_MG1655_319_IW_UV5_WT_tc/"
]

3-element Vector{String}:
 "../processing/growth_curves_plate_reader/20210712_r1_MG1655_319_IW_UV5_WT_tc/"
 "../processing/growth_curves_plate_reader/20210713_r2_MG1655_319_IW_UV5_WT_tc/"
 "../processing/growth_curves_plate_reader/20210714_r3_MG1655_319_IW_UV5_WT_tc/"

In [18]:
outputs = Glob.glob.(folder_list .* "output/*_gp_per_well.csv")

3-element Vector{Vector{String}}:
 ["../processing/growth_curves_plate_reader/20210712_r1_MG1655_319_IW_UV5_WT_tc/output/20210712_r1_gp_per_well.csv"]
 ["../processing/growth_curves_plate_reader/20210713_r2_MG1655_319_IW_UV5_WT_tc/output/20210713_r2_gp_per_well.csv"]
 ["../processing/growth_curves_plate_reader/20210714_r3_MG1655_319_IW_UV5_WT_tc/output/20210714_r3_gp_per_well.csv"]

In [19]:
df = CSV.read(outputs[1], DataFrame)
first(df, 5)

,time_min,temp_C,OD600,strain,well,media,pos_selection,promoter
,Float64,Float64,Float64,String7,String3,String15,String15,String7
1,24.1667,37.1,0.042,MG1655,A02,M9_0.5%_glucose,0_µg/ml_tc,MG1655
2,49.1667,37.0,0.044,MG1655,A02,M9_0.5%_glucose,0_µg/ml_tc,MG1655
3,74.1667,37.1,0.046,MG1655,A02,M9_0.5%_glucose,0_µg/ml_tc,MG1655
4,99.1667,37.1,0.048,MG1655,A02,M9_0.5%_glucose,0_µg/ml_tc,MG1655
5,124.167,37.0,0.051,MG1655,A02,M9_0.5%_glucose,0_µg/ml_tc,MG1655


In [20]:
names(df)

23-element Vector{String}:
 "time_min"
 "temp_C"
 "OD600"
 "strain"
 "well"
 "media"
 "pos_selection"
 "promoter"
 "volume_marker"
 "date"
 "run_number"
 "blank_val"
 "OD_sub"
 "logistic_growth_rate"
 "logistic_carrying_capacity"
 "gp_OD600"
 "log_OD600"
 "gp_OD600_std"
 "log_OD600_std"
 "gp_growth_rate"
 "gp_growth_rate_std"
 "gp_doubling_time"
 "gp_doubling_time_std"

In [22]:
strains = df.strain |> unique
df[!, :tc] = map(x -> parse(Float64, split(x, '_')[1]), df.pos_selection);

In [ ]:
fig = Figure(resolution=(600, 2000))


for (j, strain) in enumerate(strains)
    ax = Axis(fig[j, 1])
    ax.title = strain
    ax.xlabel = "time_min"
    ax.ylabel = "OD600"
    for (i, tc) in enumerate(df.tc |> unique)
        sub_df = df[(df.strain .== strain) .& (df.tc .== tc), :]
        for well in  df.well |> unique
            x = sub_df[(sub_df.well .== well), "time_min"]
            y = sub_df[(sub_df.well .== well), "OD600"]
            lines!(fig[j, 1], x, y, color = colors[i], label="$tc") 
        end
    end
    axislegend("tc [µg/ml]", unique=true, nbanks=2, position=:lt)
end


fig

## Infer Maximum Growth rate

### Gaussian process

In [3]:
gp_stan = open("gp_stan.txt") do file
    read(file, String)
end;

In [ ]:
data = Dict(
    "N" => length(x_list[1]),
    "t"=> x_list[1],
    "y"=> y_list[1], 
    "N_predict"=> length(x_list[1]),  
    "t_predict"=> x_list[1], 
    "alpha_param"=> [0, 1], 
    "sigma_param"=> [0, 1], 
    "rho_param"=> [1000, 50], 
)

analyzed_df = DataFrame(strain=String[], well=String[], tc=Float64[], λ_max=Float64[])#, dy=[])

for (j, strain) in enumerate(["MG1655"])#enumerate(strains)
    for (i, tc) in enumerate(df.tc |> unique)
        sub_df = df[(df.strain .== strain) .& (df.tc .== tc), :]
        for well in  sub_df.well |> unique
            x = sub_df[(sub_df.well .== well), "time_min"]
            y = sub_df[(sub_df.well .== well), "OD600"]
            
            data["t"] = x
            data["y"] = y
            
            sm = SampleModel("gaussian_process", gp_stan)
            rc = stan_sample(sm; data, num_chains=2);
            st = read_samples(sm)
            dy = filter(x -> occursin("dy",String(x)), names(st))
            mean_dy = [mean(st[x]) for x in dy]
            _df = DataFrame(
                strain=String[strain], 
                well=String[well], 
                tc=Float64[tc], 
                λ_max=Float64[maximum(mean_dy)], 
                #dy=Vector{Vector{Float64}}[mean_dy]
            )
            append!(analyzed_df, _df)
        end
    end
end

In [ ]:
strain_color = Dict(
    "MG1655" => ColorSchemes.BuPu_6[1],
    "3.19" => ColorSchemes.BuPu_6[2],
    "IW" => ColorSchemes.BuPu_6[3],
    "WT" => ColorSchemes.BuPu_6[4],
    "UV5" => ColorSchemes.BuPu_6[5],
)
fig = Figure(resolution=(600,400))
ax = Axis(fig[1, 1])
ax.xlabel = "tc [µg/ml]"
ax.ylabel = "λ [1/min]"

scatter!(ax,
    analyzed_df.tc, 
    analyzed_df.λ_max, 
    color=map(x-> strain_color[x], analyzed_df.strain), 
    strokecolor="black",
    strokewidth=1
)
fig

In [ ]:
strain_color = Dict(
    "MG1655" => ColorSchemes.BuPu_6[1],
    "3.19" => ColorSchemes.BuPu_6[2],
    "IW" => ColorSchemes.BuPu_6[3],
    "WT" => ColorSchemes.BuPu_6[4],
    "UV5" => ColorSchemes.BuPu_6[5],
)

analyzed_df[!, "λ/λ_0"] = analyzed_df.λ_max / mean(analyzed_df[(analyzed_df.strain .== "MG1655") .& (analyzed_df.tc .==0), "λ_max"])

fig = Figure(resolution=(600,400))
ax = Axis(fig[1, 1])
ax.xlabel = "tc [µg/ml]"
ax.ylabel = "λ/λ_0"

scatter!(ax,
    analyzed_df.tc, 
    analyzed_df[!, "λ/λ_0"], 
    color=map(x-> strain_color[x], analyzed_df.strain), 
    strokecolor="black",
    strokewidth=1
)
fig

In [ ]:
for (i, j) in collect(Iterators.product([0, 1], [3,5]))
    println(i, j)
end